In [1]:
import numpy as np
import csv
import GPy
from matplotlib import pyplot as plt
import pandas as pd
import warnings
warnings.simplefilter("error", RuntimeWarning)

In [2]:
def cnt_rmse(y, y_hat):
    return np.sqrt(((y_hat.flatten()-y.flatten())**2).mean())

In [3]:
fname = '/home/chun/Desktop/gpr_testing/parkinsons_telemonitoring/parkinsons_updrs.data'


df = pd.read_csv(fname, header=0)
X_df = df.drop(df.columns[[4,5]], axis=1)
Y_df = df.iloc[:,4]

# print(X_df.head(5))
# Y_df.head(5)

In [4]:
X = X_df.values
Y = np.reshape(Y_df.values, (len(X),1))
X.shape

(5875, 20)

In [ ]:
kernel = GPy.kern.RBF(input_dim = 20, variance = 1., lengthscale = 1., ARD = True)
# kernel = GPy.kern.RBF(input_dim = 20, variance = 1., lengthscale = 1.)
optimize = True
# optimize = False

data_idx = np.arange(len(X))
np.random.shuffle(data_idx)

test_size = int(np.floor(X.shape[0]/10))
split_idx = [test_size*i for i in range(1,10)]
test_idx_list = np.split(data_idx, split_idx)

train_size = 0
train_size = 4000
# train_size = 3000
# train_size = 2000
# train_size = 1000
# train_size = 500
# train_size = 300
# train_size = 100

err_list = []
for test_idx in test_idx_list:
    all_idx = np.arange(len(X))
    tmp_train_idx = np.delete(all_idx, test_idx)
    test_X = X[test_idx]
    if train_size == 0:
        new_train_idx = tmp_train_idx
        print('rrrr')
    else:
        np.random.shuffle(tmp_train_idx)
        new_train_idx = tmp_train_idx[:train_size]

    new_X = X[new_train_idx]
    print(test_X.shape)
    print(new_X.shape)
    
    # normalize y 
    Y_mean = Y - Y[new_train_idx].mean()
    test_y = Y_mean[test_idx]
    new_y = Y_mean[new_train_idx]
    
    try:
#         m = GPy.models.GPRegression(new_X, new_y, kernel, normalizer=True)
        m = GPy.models.GPRegression(new_X, new_y, kernel, normalizer=False)
    except RuntimeWarning as e:
        print(e, '0')
#         continue
    if optimize:
        try:
            m.optimize(max_iters=50, optimizer="lbfgs")
    #         m.optimize(max_iters=100)
        except RuntimeWarning as e:
            print(e, '1')
#             continue
    try:
        y_hat = m.predict(test_X)[0]
    except RuntimeWarning as e:
        print(e, '2')
#         continue
    try:
        err = cnt_rmse(y_hat, test_y)
    except RuntimeWarning as e:
        print(e, '3')
#         continue
    
    err_list.append(err)

print(sum(err_list)/len(err_list))

(587, 20)
(4000, 20)
(587, 20)
(4000, 20)
(587, 20)
(4000, 20)
(587, 20)
(4000, 20)
(587, 20)
(4000, 20)
